# Profit分析

## 目的

予測での利益と実際の利益の差分をみてみる。

In [1]:
import datetime
import pandas as pd

In [2]:
import sys
sys.path.append('../../')
%cd '../../'

/home/tsu-nera/repo/bakuchi


In [3]:
timestamp = "2003200210"

In [4]:
def read_trades(timestamp, exchange_id):
    file_path = "reports/{}/trades/{}.csv".format(timestamp, exchange_id)
    return pd.read_csv(file_path, index_col="id", parse_dates=["datetime"])

In [5]:
cc_trades = read_trades(timestamp, "coincheck")
lq_trades = read_trades(timestamp, "liquid")

In [6]:
len(cc_trades), len(lq_trades)

(311, 311)

In [7]:
def read_ticks(timestamp, exchange_id):
    file_path = "reports/{}/exchanges/{}.csv".format(timestamp, exchange_id)
    df = pd.read_csv(file_path, index_col="timestamp", parse_dates=["timestamp"])
    df["request_tick"] = df["tick_timestamp"]
    df["response_tick"] = df.index
    df = df.drop(["tick_timestamp"], axis=1)
    return df

In [8]:
cc_ticks = read_ticks(timestamp, "coincheck")
lq_ticks = read_ticks(timestamp, "liquid")

In [9]:
cc_ticks.head(5)

,bid,ask,request_tick,response_tick
timestamp,,,,
2020-03-20 02:10:49,688935.0,689702.0,2020-03-20 02:10:49,2020-03-20 02:10:49
2020-03-20 02:10:51,689474.0,689761.0,2020-03-20 02:10:51,2020-03-20 02:10:51
2020-03-20 02:10:52,689474.0,689761.0,2020-03-20 02:10:52,2020-03-20 02:10:52
2020-03-20 02:10:53,689474.0,689761.0,2020-03-20 02:10:53,2020-03-20 02:10:53
2020-03-20 02:10:54,689474.0,689761.0,2020-03-20 02:10:54,2020-03-20 02:10:54


In [32]:
trades = []
amount = 0.0065

for row in cc_trades.iterrows():
    cc_item = row[1]
    cc_side = cc_item["side"]
    cc_timestamp = cc_item["datetime"]
    # cc_amount = cc_item["amount"]
    cc_actual_price = cc_item["price"]
    cc_actual_rate = cc_item["rate"]
        
    try:
        cc_tick = cc_ticks.loc[cc_timestamp]
    except Exception:
        timestamp = cc_timestamp + datetime.timedelta(seconds=1)
        cc_tick = cc_ticks.loc[timestamp]
        
    if cc_side == "sell":
        cc_tick_rate = cc_tick.bid
    else:
        cc_tick_rate = cc_tick.ask
        
    cc_expected_price = cc_tick_rate * amount
    
    if cc_side == "buy":
        cc_expected_price = cc_expected_price * -1
        cc_actual_price = cc_actual_price * -1
    
    trade = {
        "timestamp": cc_timestamp, 
        "cc_side": cc_side,
        "cc_expected_rate": cc_tick_rate,
        "cc_actual_rate": cc_actual_rate,
        "cc_expected_price": cc_expected_price,
        "cc_actual_price": cc_actual_price
    }
    
    trades.append(trade)

In [33]:
profit_df = pd.DataFrame(trades)
profit_df = profit_df.set_index("timestamp")

In [34]:
profit_df

,cc_side,cc_expected_rate,cc_actual_rate,cc_expected_price,cc_actual_price
timestamp,,,,,
2020-03-20 02:10:49,sell,688935.0,689117.0,4478.0775,4479.261
2020-03-20 02:30:22,buy,693729.0,695549.0,-4509.2385,-4508.994
2020-03-20 02:30:31,sell,694135.0,693823.5,4511.8775,4510.108
2020-03-20 02:32:43,buy,694528.0,695804.0,-4514.4320,-4513.997
2020-03-20 02:32:48,sell,695118.0,695438.0,4518.2670,4520.347
...,...,...,...,...,...
2020-03-20 23:41:46,sell,736319.0,736085.0,4786.0735,4784.552
2020-03-20 23:45:05,buy,736408.0,736408.0,-4786.6520,-4776.998
2020-03-20 23:45:49,sell,737891.0,737696.0,4796.2915,4794.396
